In [235]:
# Imports 
import pandas as pd
import numpy as np
import openpyxl

def all_in_one_sequencer(location, x='DESCRIPTION',column_headers=22,sheet_name = 'Activity',column_letter = 'C'):
    # Load the Excel file
    def process_column(location, sheet_name, column_letter):
        # Load the workbook and the specified sheet
        workbook = openpyxl.load_workbook(location)
        sheet = workbook[sheet_name]
        
        # Iterate through the entire column
        for cell in sheet[column_letter]:
            # Check if the cell's fill color is black (000000)
            if cell.fill.fgColor.rgb == '00000000':  # 'FF' prefix denotes ARGB format in openpyxl
                #print(cell)
                # Remove trailing and leading spaces
                if isinstance(cell.value, str):
                    cell_value = cell.value.strip()
                    cell.value = cell_value.lstrip()
        
        # Save the workbook
        workbook.save(location)
        
    
    def presets(location,column_headers=22):
    
        # Load the Excel file
        df = pd.read_excel(location)
        
        # Set the row at index `column_headers` as the new column names
        new_columns = df.iloc[column_headers]
        
        # Update the column names
        df.columns = new_columns
        
        # Drop the row at index `column_headers`
        df = df.drop(column_headers)
        
        # Slice the DataFrame from the next row after `column_headers` to the second last row
        df = df[column_headers+1:len(df)-2]
        
        # Reset the index to start from 0
        df = df.reset_index(drop=True)
        
        return df


    # Function to determine the indentation level of a string
    def indent_level(a):
        return len(a) - len(a.lstrip())

    # Function to assign hierarchy levels based on indentation
    def assign_hierarchy(df, x):
        numbers = []
        for i in df[x]:
            numbers.append(indent_level(i) // 2)
        df['Hierarchy'] = numbers
        return df

    # Process each row in the dataframe to build the hierarchy
    def concatenator(df, x):
        # Initialize variables for processing the hierarchy
        cnt = -1
        hierarchy = []
        largest = []
        concat = []
        for i in df['Hierarchy']:
            cnt += 1
            # A heading is encountered
            if i > 0:
                concat.append('-1')
                # Check if this level of heading hierarchy has been encountered before; if not, append the new level
                if not largest or i > max(largest):
                    largest.append(i)
                    hierarchy.append(' ' + df[x][cnt])
                # If it has been encountered before, delete any following hierarchy and replace the same level hierarchy with newly encountered heading
                if i in largest:
                    index = largest.index(i)
                    hierarchy[index] = ' ' + df[x][cnt]
                    del hierarchy[index+1:]
                    del largest[index+1:]
            # For a normal record
            elif i == 0:
                concat.append(' '.join(hierarchy) + ' ' + df[x][cnt])
        df['Concat'] = concat    

    # Function to order the DataFrame based on the hierarchy
    def order(df):
        ord_lst = []
        cnt = 0 
        for j in range(len(df['Hierarchy'])):
            item = df['Hierarchy'][j]
            if item == 0:
                cnt += 1
                ord_lst.append(cnt)
            else:
                cnt = 0
                ord_lst.append(0)
        df['Order'] = ord_lst

    #presets
    df = presets(location)
    
    # Example usage:
    process_column(location, sheet_name, column_letter)

    # Assign hierarchy levels to the dataframe
    df = assign_hierarchy(df, x)
    # Concatenate based on the hierarchy
    concatenator(df, x)

    # Order the dataframe
    order(df)
    
    # Return the final DataFrame
    return df

In [236]:
# Example usage
df_result = all_in_one_sequencer('all-projects/2024-6-27-T-6-20-35.370-SA230003-27-Jun-2024.xlsx')

In [247]:
# Classifier without NLP 
cnt = 0
CNT = -1 
lst = ['Design & Engineering',  'Inspection & Delivery','Testing, Commissioning & Energization', 'Testing & Commissioning', 'Construction & Installation']
lst = [element.lower() for element in lst]
for i in df_result['Concat']:
    CNT +=1
    i = i.lower()
    if i!= '-1' and df_result['Order'][CNT]==1:
        if not any(j in i for j in lst):
            print("Errors are Possible:")
            print(i)
            cnt +=1

Errors are Possible:
 major project milestones
Errors are Possible:
   preliminary design and approval design consultant pq approval
Errors are Possible:
   base design and engineering civil preliminary design submission & approval
Errors are Possible:
   detailed design and engineering submission of electrical schemes & approval
Errors are Possible:
   equipment & materials for main bsp      control relay and protection panels po placement control relay and protection panels
Errors are Possible:
   equipment & materials for main bsp      13.8kv grounding transformer po placemen 13.8kv grounding transformer
Errors are Possible:
   equipment & materials for main bsp      free standing cts and ring type cts po placement free standing cts and ring type cts
Errors are Possible:
   equipment & materials for main bsp      energy metering system and ami panels po placemen  energy metering system and ami panels
Errors are Possible:
   equipment & materials for main bsp      tfr,fl and dsm syst

In [248]:
cnt

50